In [28]:
import tkinter as tk
import threading
import time
from PIL import Image, ImageTk
import numpy as np

root = tk.Tk()


def center_window(root, width, height):
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    root.geometry(f'{width}x{height}+{x}+{y}')

root.title("8_Queen")
center_window(root, 960, 640)
root.configure(bg='#161512')
root.resizable(False, False)



class Point_value:
    def _init_(self, x, y):
        self.x = x
        self.y = y

Showing_board = []
solve_sum = 0
#Running_board = []

Q_img = Image.open("LightQueen.png")
Q_img = Q_img.convert("RGBA")
Q_img = Q_img.resize((56, 56), Image.LANCZOS)
Q_img_show = ImageTk.PhotoImage(Q_img)


BOARD_SIZE = 8
chess_board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
img_list = []

def is_safe(board, row, col):
    for i in range(col):
        if board[row][i] == 1:
            return False
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row, 8, 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    return True



def solve_queen(board, col):
    global solve_sum
    if col >= 8:
        string_result = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == 1:
                    string_result.append(chr(j + 65))
                    string_result.append(chr(8 - i + 48))
                    string_result.append(' ')
        string_get = ''.join(string_result)
        solve_sum = solve_sum + 1
        list_solution.insert(tk.END, string_get)
        return
    for i in range(8):
        if is_safe(board, i, col):
            board[i][col] = 1
            color = '#f0d9b5' if (i + col) % 2 == 0 else '#b58863'
            label = tk.Label(root, image=Q_img_show, bg = color)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            solve_queen(board, col + 1)
            board[i][col] = 0
            label.destroy()
            label = tk.Canvas(root, width=60, height=60, bg=color, highlightthickness=0)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            label.destroy()

def print_result(result_string):
    side = 480
    x = 50
    y = 50
    border_canvas = tk.Canvas(root, width=(side + 8), height=(side + 8))
    border_canvas.place(x=x - 6, y=y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, side, side,
                                   outline=border_color, width=border_thickness)
    for row in range(8):
        for col in range(8):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(root, width=60, height=60, bg=color, highlightthickness=0)
            square.place(x=x + col * 60, y=y + row * 60)
            
    print_point = [0 for _ in range(BOARD_SIZE)]
    for index in range(0, len(result_string), 3):
        point_place = result_string[index:index+3]
        print_point[index // 3] = [ord(point_place[0]) - 65, 8 - (ord(point_place[1]) - 48)]
    
    def get_color(point):
        color = '#f0d9b5' if (point[0] + point[1]) % 2 == 0 else '#b58863'
        return color

    for select_point in range(8):
        label = tk.Label(root, image=Q_img_show, bg = get_color(print_point[select_point]))
        label.place(x=Showing_board[print_point[select_point][1]][print_point[select_point][0]].x, 
                    y=Showing_board[print_point[select_point][1]][print_point[select_point][0]].y)
    root.update_idletasks()
        




def finished_signal():
    signal_window = tk.Toplevel(root)
    signal_window.title("已完成")
    center_window(signal_window, 160, 50)
    signal_window.resizable(False, False)
    label = tk.Label(signal_window, text="已完成：共" + str(solve_sum) + "个解", 
             bg='#161512',  fg='#cacaca',  font=("黑体", 12, 'bold'), 
             width=20,  height=5)
    signal_window.grid_rowconfigure(0, weight=1)
    signal_window.grid_columnconfigure(0, weight=1)
    label.grid(row=0, column=0, sticky="nsew")

def solving():
    solve_queen(chess_board, 0)
    buttons[1].config(state=tk.NORMAL)
    finished_signal()


def run_order():
    buttons[0].config(state=tk.DISABLED)
    buttons[1].config(state=tk.NORMAL)
    solving()

def reset_order():
    chess_board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    global solve_sum
    solve_sum = 0
    buttons[0].config(state=tk.NORMAL)
    buttons[1].config(state=tk.DISABLED)
    list_solution.delete(0, tk.END)

    side = 480
    x = 50
    y = 50
    border_canvas = tk.Canvas(root, width=(side + 8), height=(side + 8))
    border_canvas.place(x=x - 6, y=y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, side, side,
                                   outline=border_color, width=border_thickness)
    square_size = 60
    for row in range(8):
        row_point = []
        for col in range(8):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(root, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=x + col * square_size, y=y + row * square_size)

def show_result(evt):
    w = evt.widget
    indices = w.curselection()
    if indices:
        for index in indices:
            value = w.get(index)
            print_result(value)



def create_chessboard(parent, x, y, side, square, saver_board):
    border_canvas = tk.Canvas(parent, width=(side + 8), height=(side + 8))
    border_canvas.place(x=x - 6, y=y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, side, side,
                                   outline=border_color, width=border_thickness)
    square_size = square
    for row in range(8):
        row_point = []
        for col in range(8):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(parent, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=x + col * square_size, y=y + row * square_size)
            row_point.append(Point_value())
            row_point[col].x = x + col * square_size
            row_point[col].y = y + row * square_size
        saver_board.append(row_point)
    for row in range(8):
        label = tk.Label(parent, text = chr(8 - row + 48))
        label.configure(bg="#161512", fg="#FFFFFF", font=("Arial", 12, 'bold'))
        label.place(x = x - 30, y = y + 20 + row * square_size)
    for col in range(8):
        label = tk.Label(parent, text = chr(col + 65))
        label.configure(bg="#161512", fg="#FFFFFF", font=("Arial", 12, 'bold'))
        label.place(x = x + 20 + col * square_size, y = y + side + 15)


create_chessboard(root, 50, 50, 480, 60, Showing_board)
#create_chessboard(root, 650, 350, 176, 22, Running_board)

buttons = [tk.Button(root, text="求解", command=run_order), 
           tk.Button(root, text="重置", command=reset_order, state=tk.DISABLED), ]
for button in buttons:
    button.configure(compound='left', bg="#51504d", fg="#cacaca", font=("黑体", 12, 'bold'), height=1, width=8)
buttons[0].place(x=670, y=280)
buttons[1].place(x=770, y=280)
list_solution = tk.Listbox(root)
list_solution.configure(bg="#e1cfc1", fg="#302b24", font=("Arial", 12, 'bold'), height=6, width=25)
list_solution.place(x=635, y=60)
list_solution.bind('<<ListboxSelect>>', show_result)

root.mainloop()